# **<span style="font-family: 'Palatino Linotype', serif;"> 📜✨🔐 A senha de tamanho variável </span>**
----
*<span style="font-family: 'Angilla Tattoo'"> "Forjada no fulgor das runas líquidas, a Senha de Tamanho Variável respondia apenas à vontade arcana da Maga Patolina."🪄📡📘 </span>*



<div align="center">
    <img src = "Maga senha.png" alt = "Maga senha" width = 250>
</div>

----
 **Objetivo:** Nesse notebook, iremos resolver o problema da senha variável, sem fornecer o tamanho da senha para que a população seja formada. Para isso, iremos adapatar nossa função objetivo, que irá levar em conta tanto a distância entre os caracteres da senha possível com a senha real, quanto a distância entre o tamanho (quantidade de caracteres), entre senha possível e senha real.
 
 ----

In [222]:
# Importar biblioteca

import random
from string import ascii_lowercase, ascii_uppercase, digits

Adicinamos o caracter "\0" aos caracteres possíveis, ele indica um caracter ausente. Esse caracter não é printando pelo Python.

In [223]:
senha = "oi\0Cassar"
print(senha)
senha_2 = "tchau!\0"
print(senha_2)

oi Cassar
tchau! 


In [224]:
AUSENTE = "\0"
candidato_possivel = ascii_lowercase + ascii_uppercase + digits + AUSENTE
print(candidato_possivel)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 


In [225]:
# Função criar gene
def cria_gene_senha(candidato_possivel):

    candidato = random.choice(candidato_possivel) 
    return candidato

In [226]:
# Função cria candidato

def cria_candidato_senha(tamanho_max, candidato_possivel):
    candidato = []
    
    for i in range(tamanho_max):
        gene = cria_gene_senha(candidato_possivel)
        candidato.append(gene)

    return candidato

In [227]:
# Função criar população
def populacao_senha(tamanho_populacao, tamanho_max, candidato_possivel):
    
    populacao = []

    for _ in range(tamanho_populacao):
        populacao.append(cria_candidato_senha(tamanho_max, candidato_possivel))

    return populacao


In [228]:
def funcao_objetivo_senha(candidato, senha_verdadeira):
    
    distancia_gene = 0
    distancia_tamanho = abs(len(candidato) - len(senha_verdadeira))
    
    limite = min(len(candidato), len(senha_verdadeira)) # encontra o valor mínimo de carateres entre o candidato e a senha verdadeira
    for i in range(limite):
        num_candidato = ord(candidato[i])
        num_verdadeira = ord(senha_verdadeira[i])
        distancia_gene += abs(num_candidato - num_verdadeira)

    fitness = distancia_gene + distancia_tamanho
    return fitness


def funcao_objetivo_pop_senha(populacao, senha_verdadeira):
    
    fitness = []

    for individuo in populacao:
        fitness.append(funcao_objetivo_senha(individuo, senha_verdadeira))

    return fitness

In [229]:
# Função de seleção

def selecao_torneio_min(populacao, fitness, tamanho_torneio):

    selecionados = []

    for _ in range(len(populacao)):
        sorteados = random.sample(populacao, tamanho_torneio)

        fitness_sorteados = []
        for individuo in sorteados:
            indice_individuo = populacao.index(individuo)
            fitness_sorteados.append(fitness[indice_individuo])

        min_fitness = min(fitness_sorteados)
        indice_min_fitness = fitness_sorteados.index(min_fitness)
        individuo_selecionado = sorteados[indice_min_fitness]

        selecionados.append(individuo_selecionado)

    return selecionados

In [230]:
# Função de cruzamento

def cruzamento_ponto_simples_senha(pai, mae, chance_de_cruzamento, SENHA):
  
    index_pai_max = len(pai)
    index_mae_max = len(mae)
            
            
    if random.random() < chance_de_cruzamento:
        # verificar o valor mínimo possível para os index
        limite = min(index_mae_max, index_pai_max)
        if limite <= 1:
            corte = 1
        else:
            corte = random.randint(1, limite - 1)
            
        filho1 = pai[:corte] + mae[corte:limite]
        filho1 += [AUSENTE] * (len(SENHA) - len(filho1))
        filho2 = mae[:corte] + pai[corte:limite]
        filho2 += [AUSENTE] * (len(SENHA) - len(filho2))
        return filho1, filho2
    else:
        return pai, mae

In [231]:
# Função mutação simples
def mutacao_simples(populacao, chance_de_mutacao, valores_possiveis):
   
    for individuo in populacao:
        if random.random() < chance_de_mutacao:

            index_individuo = len(individuo) - 1

            if index_individuo == 0:
                continue

            gene = random.randint(0, index_individuo) # escolhe o índice que será mutado
            valor_gene = individuo[gene] # armazena o valor do gene    
            valores_sorteio = set(valores_possiveis) - {valor_gene} # cria a lista dos valores que podem ser sorteados (qualquer valor possível excluindo o valor do próprio gene)
            individuo[gene] = random.choice(list(valores_sorteio)) 

In [232]:
senha = "Robertinha2705"

AUSENTE = '\0'
SENHA = list(senha) 
SENHA = SENHA  + ([AUSENTE] * (30 - len(SENHA)))

CARACTERES_POSSIVEIS = ascii_lowercase + ascii_uppercase + digits + AUSENTE

TAMANHO_POPULACAO = 2_000
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.3
TAMANHO_TORNEIO = 3
TAMANHO_MAXIMO_SENHA = 30

print(SENHA)

['R', 'o', 'b', 'e', 'r', 't', 'i', 'n', 'h', 'a', '2', '7', '0', '5', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00']


In [233]:
populacao = populacao_senha(TAMANHO_POPULACAO, len(SENHA), CARACTERES_POSSIVEIS)

In [234]:
menor_fitness_geral = float("inf")
geracao = 0

while menor_fitness_geral != 0:
    
    # Seleção
    fitness = funcao_objetivo_pop_senha(populacao, SENHA)        
    selecionados = selecao_torneio_min(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = cruzamento_ponto_simples_senha(pai, mae, CHANCE_DE_CRUZAMENTO, SENHA)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    mutacao_simples(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))

    # Encerramento
    populacao = proxima_geracao
    geracao += 1
    
    fitness = funcao_objetivo_pop_senha(populacao, SENHA)
    menor_fitness_observado = min(fitness)
    
    
    if menor_fitness_observado < menor_fitness_geral:
        menor_fitness_geral = menor_fitness_observado
        indice = fitness.index(menor_fitness_observado)
        candidato = populacao[indice]
        print(geracao, "".join(candidato).rstrip(AUSENTE)) 

1 oLwfuPGk4Y99zFXMFh6j216Q0O S74
2 dhulwo2SgeJ9xS44VL6j216Q0O S74
3 OrWgozqT0EPr3z  HJ5 8 pu81S5PP
4 VoKNszQikj4JBsRD Zd7777o599 2O
5 VoKNszej1f3919Jb BeE9 ueN21  y
6 RpqbpqXsYfRL29Jb B3E jgQ0O S74
7 JjCDxjZVBPm1C1S21 AB 1 EB21  y
8 OrNgaqXsYfm1C1S21 AB 1 E0O 274
9 WRWbpqxtjH6275 32kx  0 0J 8344
10 XXfmyshrvt7919Jb  AB 1 E0O 274
11 OrDfjeneuiBFC1  HJ5 8 p3j 1
13 OrWgoshrvt7419   B3B 1 EB218 F
14 Xtfmyshrvt774j d    8 2 w 6  8
15 OrWgoshrvt7919 d      2 0Y 27A
16 OrWgoshrvt7419   B3B 0 0J 0  4
17 OrWgoshrvt7919 d     0 0  8S74
18 OrWabxhrvt7919  1    0 Q021  4
19 OrWgoshrvt7919 d     0 0  8
21 RpqbpqXsYf2275       1 0  6 D
22 OrWgoshrvt7419        20  1
24 XXfmyshqlf3919        2   8
25 TxbUyjhqlf3919        2   8
26 XXfmyvhqlfB919       0
28 Rpqbyshqlf3K19        2
29 Rpqbyshqlc3UC1
31 OrWbtqgkYg2275
33 Tjdgoshaga3919
36 Rpbfrshrgf3919
39 Rpbfrshrla3919
41 Rpbfrshkia3919
50 Rpbfrshrga3815
54 Rpbfrshqga3815
57 Rpbfrshlga3815
58 Rpbfrsioga3815
65 Robfrshmga3715
66 Robfrsimga3715
72 Robfr

-----

### 📊 Conclusão:

Nesse notebook, exploramos o funcionamento dos algoritmos genéticos no contexto de uma senha variável. Nesse sentido, foi possível explorar a funcionalidade da função objetivo, e como podemos adaptar seus parâmetros as necessidades de cada problema. Nesse caso, além de calcular o quão distante o caracter sorteado estava do caracter real, também consideramos o quão distante o tamanho da senha sorteada está do tamanho da senha real. Ambos os resultados são utilizados como parâmetros para avaliar o quão bom é um candidato, permitindo que encontremos a senha de um usuário mesmo sem saber qual seu tamanho original. 

----
### Agradeciemntos:

Esse código foi feito com inspiração no código do colega Emanuel Piveta Pozzobon.

----
### 📚 Referências:

1. ASCII Table. Disponível em: <https://www.ascii-code.com/>. Acesso em 25 de mai. de 2025. 

4. OPENAI. ChatGPT. 2025. Disponível em: <https://chatgpt.com/share/6835c114-fbb0-8005-8150-bc0df8241484>. Acesso em: 25 mai. 2025.